<a href="https://colab.research.google.com/github/zackm-solid/Human-vs-AI-Analyst/blob/main/gemini/colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# Load the file into a DataFrame
try:
    df = pd.read_csv('/home/employment_data_final.csv')

    print("--- 1. DataFrame Info() ---")
    print("Checking column names, data types, and non-null counts:\n")
    # .info() prints its output directly, so we just call it
    df.info()

    print("\n" + "="*60 + "\n")

    print("--- 2. DataFrame Describe() (Numerical) ---")
    print("Statistical summary for numerical columns:\n")
    # .describe() returns a DataFrame, so we print it
    print(df.describe())

    print("\n" + "="*60 + "\n")

    print("--- 3. DataFrame Describe() (Categorical/Object) ---")
    print("Statistical summary for non-numerical columns (like 'Title'):\n")
    # This specifically checks string/object columns
    print(df.describe(include=['object']))

except FileNotFoundError:
    print("Error: The file 'employment_data_final.csv' was not found.")
    print("Please make sure the file is in the same directory as your script or provide the full path.")
except Exception as e:
    print(f"An error occurred: {e}")

--- 1. DataFrame Info() ---
Checking column names, data types, and non-null counts:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785 entries, 0 to 784
Data columns (total 15 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   SOC Code                                         785 non-null    object 
 1   title                                            785 non-null    object 
 2   ai_applicability_score                           785 non-null    float64
 3   Employment 2024 (Thousands)                      785 non-null    object 
 4   Employment 2034 (Thousands)                      785 non-null    object 
 5   Employment Change, 2024-2034 (Thousands)         785 non-null    float64
 6   Employment Percent Change, 2024-2034             785 non-null    float64
 7   Occupational Openings, 2024-2034 Annual Average  785 non-null    float64
 8   Median Annual Wage 2024  

In [6]:
import plotly.express as px
import numpy as np
import pandas as pd

# --- Data Cleaning and Preparation ---

# 1. Clean 'Median Annual Wage 2024' column
def clean_wage(wage_str):
    if pd.isna(wage_str):
        return np.nan
    if isinstance(wage_str, str):
        # Remove commas
        wage_str = wage_str.replace(',', '')
        # Handle the '>=XXX' format by taking the numerical part
        if wage_str.startswith('>='):
            return float(wage_str[2:])
        try:
            return float(wage_str)
        except ValueError:
            return np.nan # Return NaN if conversion fails
    return wage_str # Return as is if already a number or None

df['Median Annual Wage 2024_cleaned'] = df['Median Annual Wage 2024'].apply(clean_wage)

# Fill any remaining NaN values in the cleaned wage column with the median wage
# This ensures all data points can be colored and avoids dropping rows.
median_wage_value = df['Median Annual Wage 2024_cleaned'].median()
df['Median Annual Wage 2024_cleaned'] = df['Median Annual Wage 2024_cleaned'].fillna(median_wage_value)

# 2. Prepare data for bubble size (absolute value of Employment Change)
df['Employment Change Absolute (Thousands)'] = df['Employment Change, 2024-2034 (Thousands)'].abs()

# --- Calculate means for reference lines ---
mean_ai_applicability_score = df['ai_applicability_score'].mean()
mean_employment_percent_change = df['Employment Percent Change, 2024-2034'].mean()

# --- Create the Interactive Bubble Chart ---
fig = px.scatter(df,
                 x='ai_applicability_score',
                 y='Employment Percent Change, 2024-2034',
                 size='Employment Change Absolute (Thousands)', # Use the absolute change for bubble size
                 color='Median Annual Wage 2024_cleaned',      # Use the cleaned wage for color
                 hover_name='title',                             # Display 'title' prominently on hover
                 hover_data={
                     'ai_applicability_score': ':.2f',
                     'Employment Percent Change, 2024-2034': ':.2f%',
                     'Employment Change Absolute (Thousands)': ':.1f',
                     'Median Annual Wage 2024_cleaned': '$,.0f', # Format as currency
                     'title': False # Prevent 'title' from being repeated in the hover_data box
                 },
                 title='AI Growth Map',
                 labels={
                     'ai_applicability_score': 'AI Applicability Score',
                     'Employment Percent Change, 2024-2034': 'Employment Percent Change (2024-2034)',
                     'Employment Change Absolute (Thousands)': 'Employment Change (Thousands)',
                     'Median Annual Wage 2024_cleaned': 'Median Annual Wage'
                 },
                 color_continuous_scale=px.colors.sequential.OrRd, # Change to OrRd color scale
                 size_max=60 # Set a maximum bubble size for better visualization
                )

# Add dashed vertical reference line for mean AI Applicability Score
fig.add_shape(type="line",
              x0=mean_ai_applicability_score, y0=df['Employment Percent Change, 2024-2034'].min(),
              x1=mean_ai_applicability_score, y1=df['Employment Percent Change, 2024-2034'].max(),
              line=dict(color="Gray", width=2, dash="dash"),
              name="Mean AI Applicability Score",
              layer="below") # Place line behind bubbles

# Add dashed horizontal reference line for mean Employment Percent Change
fig.add_shape(type="line",
              x0=df['ai_applicability_score'].min(), y0=mean_employment_percent_change,
              x1=df['ai_applicability_score'].max(), y1=mean_employment_percent_change,
              line=dict(color="Gray", width=2, dash="dash"),
              name="Mean Employment Percent Change",
              layer="below") # Place line behind bubbles

fig.update_layout(showlegend=True) # Ensure the color legend is visible
fig.show()

In [7]:
import re

# Define the list of keywords
keywords = ['Analyst', 'Data', 'Specialist', 'Operations']

# Create a regex pattern for case-insensitive matching of any keyword
# The '|' acts as an OR operator in regex
# re.IGNORECASE makes the search case-insensitive
pattern = '|'.join(keywords)

# Filter the DataFrame
# The `case=False` argument ensures case-insensitive matching
# The `na=False` argument handles any potential NaN values in the 'title' column
analytics_df = df[df['title'].str.contains(pattern, case=False, na=False)].copy()

# Print the head of the new DataFrame
print("Head of analytics_df (filtered by keywords):")
print(analytics_df.head())

Head of analytics_df (filtered by keywords):
   SOC Code                            title  ai_applicability_score  \
1   11-1021  General and Operations Managers                0.121028   
43  13-1071      Human Resources Specialists                0.200716   
45  13-1075      Labor Relations Specialists                0.067259   
47  13-1082   Project Management Specialists                0.182398   
48  13-1111              Management Analysts                0.352892   

   Employment 2024 (Thousands) Employment 2034 (Thousands)  \
1                     3,712.90                    3,876.80   
43                       944.3                    1,002.70   
45                        65.4                        65.4   
47                    1,046.30                    1,105.00   
48                    1,075.10                    1,169.70   

    Employment Change, 2024-2034 (Thousands)  \
1                                      164.0   
43                                      58.4   
45   

In [8]:
import plotly.express as px

# Sort the DataFrame by 'ai_applicability_score' in descending order
analytics_df_sorted = analytics_df.sort_values(by='ai_applicability_score', ascending=False)

# Create the horizontal bar chart
fig = px.bar(analytics_df_sorted,
             x='ai_applicability_score',
             y='title',
             orientation='h', # Explicitly set orientation to horizontal
             title='Analytics Role Deep-Dive',
             labels={
                 'ai_applicability_score': 'AI Applicability Score',
                 'title': 'Job Title'
             },
             height=800 # Adjust height for better readability of many bars
            )

fig.update_layout(yaxis={'categoryorder':'total ascending'}) # Ensure sorting is applied correctly on y-axis
fig.show()

In [9]:
import plotly.express as px

# Sort the DataFrame by 'Employment Percent Change, 2024-2034' in descending order
analytics_df_growth_sorted = analytics_df.sort_values(by='Employment Percent Change, 2024-2034', ascending=False)

# Create the horizontal bar chart
fig = px.bar(analytics_df_growth_sorted,
             x='Employment Percent Change, 2024-2034',
             y='title',
             orientation='h', # Explicitly set orientation to horizontal
             title='Analytics Role Growth',
             labels={
                 'Employment Percent Change, 2024-2034': 'Employment Percent Change (2024-2034)',
                 'title': 'Job Title'
             },
             height=800 # Adjust height for better readability of many bars
            )

fig.update_layout(yaxis={'categoryorder':'total ascending'}) # Ensure sorting is applied correctly on y-axis
fig.show()

In [11]:
# Filter jobs with AI Applicability Score below the mean
least_ai_affected_jobs = df[df['ai_applicability_score'] < mean_ai_applicability_score].copy()

# Sort by 'Employment Percent Change' in descending order and get the top 20
least_ai_affected_jobs_top20 = least_ai_affected_jobs.sort_values(
    by='Employment Percent Change, 2024-2034', ascending=False
).head(20)

print("Top 20 Least AI-Affected Jobs with Highest Employment Growth:")
print(least_ai_affected_jobs_top20[['title', 'ai_applicability_score', 'Employment Percent Change, 2024-2034']])

Top 20 Least AI-Affected Jobs with Highest Employment Growth:
                                                 title  \
630                   Wind Turbine Service Technicians   
310                                Nurse Practitioners   
28                Medical and Health Services Managers   
363                      Physical Therapist Assistants   
84                                           Actuaries   
296                               Physician Assistants   
342                            Psychiatric Technicians   
345                     Ophthalmic Medical Technicians   
361                    Occupational Therapy Assistants   
63                                 Financial Examiners   
350                            Hearing Aid Specialists   
357                Home Health and Personal Care Aides   
46                                        Logisticians   
619                     Industrial Machinery Mechanics   
365                                 Massage Therapists   
10        

In [13]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.5 MB/s eta 0:00:00


### Streamlit Dashboard Code

To run this Streamlit dashboard:

1.  **Install Streamlit**: If you haven't already, install Streamlit by running `pip install streamlit`. You might also need `pandas` and `plotly` if they are not already installed in your environment.
2.  **Save the code**: Copy the Python code below and save it as a Python file (e.g., `dashboard.py`) on your local machine.
3.  **Run the app**: Open your terminal or command prompt, navigate to the directory where you saved `dashboard.py`, and run the command: `streamlit run dashboard.py`.

This will open the interactive dashboard in your web browser.

In [15]:
import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np
import re

# --- 1. Load and Prepare Data ---

# Load the data
try:
    df = pd.read_csv('/home/employment_data_final.csv')
except FileNotFoundError:
    st.error("Error: 'employment_data_final.csv' not found. Please ensure the file is in the correct directory.")
    st.stop() # Stop the app if the file is not found

# Clean 'Median Annual Wage 2024' column
def clean_wage(wage_str):
    if pd.isna(wage_str):
        return np.nan
    if isinstance(wage_str, str):
        wage_str = wage_str.replace(',', '')
        if wage_str.startswith('>='):
            return float(wage_str[2:])
        try:
            return float(wage_str)
        except ValueError:
            return np.nan
    return wage_str

df['Median Annual Wage 2024_cleaned'] = df['Median Annual Wage 2024'].apply(clean_wage)
median_wage_value = df['Median Annual Wage 2024_cleaned'].median()
df['Median Annual Wage 2024_cleaned'] = df['Median Annual Wage 2024_cleaned'].fillna(median_wage_value)

# Prepare data for bubble size (absolute value of Employment Change)
df['Employment Change Absolute (Thousands)'] = df['Employment Change, 2024-2034 (Thousands)'].abs()

# Calculate means for reference lines in AI Growth Map
mean_ai_applicability_score = df['ai_applicability_score'].mean()
mean_employment_percent_change = df['Employment Percent Change, 2024-2034'].mean()

# Filter for analytics roles (case-insensitive)
keywords = ['Analyst', 'Data', 'Specialist', 'Operations']
pattern = '|'.join(keywords)
analytics_df = df[df['title'].str.contains(pattern, case=False, na=False)].copy()

# --- 2. Create Plotly Charts ---

# A. AI vs Growth Map (Bubble Chart)
fig_bubble = px.scatter(df,
                        x='ai_applicability_score',
                        y='Employment Percent Change, 2024-2034',
                        size='Employment Change Absolute (Thousands)',
                        color='Median Annual Wage 2024_cleaned',
                        hover_name='title',
                        hover_data={
                            'ai_applicability_score': ':.2f',
                            'Employment Percent Change, 2024-2034': ':.2f%',
                            'Employment Change Absolute (Thousands)': ':.1f',
                            'Median Annual Wage 2024_cleaned': '$,.0f',
                            'title': False
                        },
                        title='AI Growth Map',
                        labels={
                            'ai_applicability_score': 'AI Applicability Score',
                            'Employment Percent Change, 2024-2034': 'Employment Percent Change (2024-2034)',
                            'Employment Change Absolute (Thousands)': 'Employment Change (Thousands)',
                            'Median Annual Wage 2024_cleaned': 'Median Annual Wage'
                        },
                        color_continuous_scale=px.colors.sequential.OrRd,
                        size_max=60)

fig_bubble.add_shape(type="line",
                     x0=mean_ai_applicability_score, y0=df['Employment Percent Change, 2024-2034'].min(),
                     x1=mean_ai_applicability_score, y1=df['Employment Percent Change, 2024-2034'].max(),
                     line=dict(color="Gray", width=2, dash="dash"),
                     name="Mean AI Applicability Score",
                     layer="below")
fig_bubble.add_shape(type="line",
                     x0=df['ai_applicability_score'].min(), y0=mean_employment_percent_change,
                     x1=df['ai_applicability_score'].max(), y1=mean_employment_percent_change,
                     line=dict(color="Gray", width=2, dash="dash"),
                     name="Mean Employment Percent Change",
                     layer="below")
fig_bubble.update_layout(showlegend=True)

# B. Analytics Role Deep-Dive (Horizontal Bar Chart)
analytics_df_sorted_ai = analytics_df.sort_values(by='ai_applicability_score', ascending=False)
fig_deep_dive = px.bar(analytics_df_sorted_ai,
                       x='ai_applicability_score',
                       y='title',
                       orientation='h',
                       title='Analytics Role Deep-Dive',
                       labels={
                           'ai_applicability_score': 'AI Applicability Score',
                           'title': 'Job Title'
                       },
                       height=600)
fig_deep_dive.update_layout(yaxis={'categoryorder':'total ascending'})

# C. Analytics Role Growth (Horizontal Bar Chart)
analytics_df_sorted_growth = analytics_df.sort_values(by='Employment Percent Change, 2024-2034', ascending=False)
fig_growth = px.bar(analytics_df_sorted_growth,
                    x='Employment Percent Change, 2024-2034',
                    y='title',
                    orientation='h',
                    title='Analytics Role Growth',
                    labels={
                        'Employment Percent Change, 2024-2034': 'Employment Percent Change (2024-2034)',
                        'title': 'Job Title'
                    },
                    height=600)
fig_growth.update_layout(yaxis={'categoryorder':'total ascending'})

# --- 3. Streamlit App Layout ---
st.set_page_config(layout="wide", page_title="AI Job Market Dashboard")
st.title("AI Job Market Analysis Dashboard")

col1, col2 = st.columns([2, 1]) # Adjust column ratio for layout

with col1:
    st.plotly_chart(fig_bubble, use_container_width=True)

with col2:
    st.plotly_chart(fig_deep_dive, use_container_width=True)
    st.plotly_chart(fig_growth, use_container_width=True)



2025-11-15 22:09:33.194 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 22:09:33.195 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 22:09:33.196 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 22:09:33.197 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 22:09:33.199 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 22:09:33.201 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 22:09:33.202 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 22:09:33.203 Please replace `use_container_width` with `width`.

`use_container_width` will be removed afte

In [ ]:
# Save the Streamlit app code to a Python file
with open('dashboard.py', 'w') as f:
    f.write('''
import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np
import re

# --- 1. Load and Prepare Data ---

# Load the data
try:
    df = pd.read_csv('/home/employment_data_final.csv')
except FileNotFoundError:
    st.error("Error: 'employment_data_final.csv' not found. Please ensure the file is in the correct directory.")
    st.stop() # Stop the app if the file is not found

# Clean 'Median Annual Wage 2024' column
def clean_wage(wage_str):
    if pd.isna(wage_str):
        return np.nan
    if isinstance(wage_str, str):
        wage_str = wage_str.replace(',', '')
        if wage_str.startswith('>='):
            return float(wage_str[2:])
        try:
            return float(wage_str)
        except ValueError:
            return np.nan
    return wage_str

df['Median Annual Wage 2024_cleaned'] = df['Median Annual Wage 2024'].apply(clean_wage)
median_wage_value = df['Median Annual Wage 2024_cleaned'].median()
df['Median Annual Wage 2024_cleaned'] = df['Median Annual Wage 2024_cleaned'].fillna(median_wage_value)

# Prepare data for bubble size (absolute value of Employment Change)
df['Employment Change Absolute (Thousands)'] = df['Employment Change, 2024-2034 (Thousands)'].abs()

# Calculate means for reference lines in AI Growth Map
mean_ai_applicability_score = df['ai_applicability_score'].mean()
mean_employment_percent_change = df['Employment Percent Change, 2024-2034'].mean()

# Filter for analytics roles (case-insensitive)
keywords = ['Analyst', 'Data', 'Specialist', 'Operations']
pattern = '|'.join(keywords)
analytics_df = df[df['title'].str.contains(pattern, case=False, na=False)].copy()

# --- 2. Create Plotly Charts ---

# A. AI vs Growth Map (Bubble Chart)
fig_bubble = px.scatter(df,
                        x='ai_applicability_score',
                        y='Employment Percent Change, 2024-2034',
                        size='Employment Change Absolute (Thousands)',
                        color='Median Annual Wage 2024_cleaned',
                        hover_name='title',
                        hover_data={
                            'ai_applicability_score': ':.2f',
                            'Employment Percent Change, 2024-2034': ':.2f%',
                            'Employment Change Absolute (Thousands)': ':.1f',
                            'Median Annual Wage 2024_cleaned': '$,.0f',
                            'title': False
                        },
                        title='AI Growth Map',
                        labels={
                            'ai_applicability_score': 'AI Applicability Score',
                            'Employment Percent Change, 2024-2034': 'Employment Percent Change (2024-2034)',
                            'Employment Change Absolute (Thousands)': 'Employment Change (Thousands)',
                            'Median Annual Wage 2024_cleaned': 'Median Annual Wage'
                        },
                        color_continuous_scale=px.colors.sequential.OrRd,
                        size_max=60)

fig_bubble.add_shape(type="line",
                     x0=mean_ai_applicability_score, y0=df['Employment Percent Change, 2024-2034'].min(),
                     x1=mean_ai_applicability_score, y1=df['Employment Percent Change, 2024-2034'].max(),
                     line=dict(color="Gray", width=2, dash="dash"),
                     name="Mean AI Applicability Score",
                     layer="below")
fig_bubble.add_shape(type="line",
                     x0=df['ai_applicability_score'].min(), y0=mean_employment_percent_change,
                     x1=df['ai_applicability_score'].max(), y1=mean_employment_percent_change,
                     line=dict(color="Gray", width=2, dash="dash"),
                     name="Mean Employment Percent Change",
                     layer="below")
fig_bubble.update_layout(showlegend=True)

# B. Analytics Role Deep-Dive (Horizontal Bar Chart)
analytics_df_sorted_ai = analytics_df.sort_values(by='ai_applicability_score', ascending=False)
fig_deep_dive = px.bar(analytics_df_sorted_ai,
                       x='ai_applicability_score',
                       y='title',
                       orientation='h',
                       title='Analytics Role Deep-Dive',
                       labels={
                           'ai_applicability_score': 'AI Applicability Score',
                           'title': 'Job Title'
                       },
                       height=600)
fig_deep_dive.update_layout(yaxis={'categoryorder':'total ascending'})

# C. Analytics Role Growth (Horizontal Bar Chart)
analytics_df_sorted_growth = analytics_df.sort_values(by='Employment Percent Change, 2024-2034', ascending=False)
fig_growth = px.bar(analytics_df_sorted_growth,
                    x='Employment Percent Change, 2024-2034',
                    y='title',
                    orientation='h',
                    title='Analytics Role Growth',
                    labels={
                        'Employment Percent Change, 2024-2034': 'Employment Percent Change (2024-2034)',
                        'title': 'Job Title'
                    },
                    height=600)
fig_growth.update_layout(yaxis={'categoryorder':'total ascending'})

# --- 3. Streamlit App Layout ---
st.set_page_config(layout="wide", page_title="AI Job Market Dashboard")
st.title("AI Job Market Analysis Dashboard")

col1, col2 = st.columns([2, 1]) # Adjust column ratio for layout

with col1:
    st.plotly_chart(fig_bubble, use_container_width=True)

with col2:
    st.plotly_chart(fig_deep_dive, use_container_width=True)
    st.plotly_chart(fig_growth, use_container_width=True)

''')

print("Streamlit dashboard code saved to dashboard.py")

# Run the Streamlit app. This command will usually print a URL to the output.
# You might need to use a tunneling service like localtunnel if you can't access it directly.
!streamlit run dashboard.py --server.port 8501 --server.enableCORS false

Streamlit dashboard code saved to dashboard.py
2025-11-15 22:10:15.264 
'server.enableXsrfProtection=true'.
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.121.55:8501

